In [ ]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

car_template = """You are an expert in automobiles. You have extensive knowledge about car mechanics, \
models, and automotive technology. You provide clear and helpful answers about cars.

Here is a question:
{query}"""

restaurant_template = """You are a knowledgeable food critic and restaurant reviewer. You have a deep understanding of \
different cuisines, dining experiences, and what makes a great restaurant. You answer questions about restaurants insightfully.

Here is a question:
{query}"""

technology_template = """You are a tech expert with in-depth knowledge of the latest gadgets, software, \
and technological trends. You provide insightful and detailed answers about technology.

Here is a question:
{query}"""

In [ ]:
car_questions = [
    "What is the difference between a sedan and an SUV?",
    "How does a hybrid car save fuel?",
    "What should I look for when buying a used car?",
]

restaurant_questions = [
    "What makes a five-star restaurant exceptional?",
    "How do I choose a good wine pairing for my meal?",
    "What are the key elements of French cuisine?",
]

technology_questions = [
    "What are the latest advancements in AI?",
    "How do I secure my home network against cyber threats?",
    "What should I consider when buying a new smartphone?",
]

In [ ]:
embeddings = OpenAIEmbeddings()

car_question_embeddings = embeddings.embed_documents(car_questions)
restaurant_question_embeddings = embeddings.embed_documents(restaurant_questions)
technology_question_embeddings = embeddings.embed_documents(technology_questions)

In [ ]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    car_similarity = cosine_similarity([query_embedding], car_question_embeddings)[0]
    restaurant_similarity = cosine_similarity(
        [query_embedding], restaurant_question_embeddings
    )[0]
    technology_similarity = cosine_similarity(
        [query_embedding], technology_question_embeddings
    )[0]

    max_similarity = max(
        max(car_similarity), max(restaurant_similarity), max(technology_similarity)
    )

    if max_similarity == max(car_similarity):
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif max_similarity == max(restaurant_similarity):
        print("Using RESTAURANT")
        return PromptTemplate.from_template(restaurant_template)
    else:
        print("Using TECHNOLOGY")
        return PromptTemplate.from_template(technology_template)


input_query = {"query": "What's the best way to improve my cars's battery life?"}
prompt = prompt_router(input_query)

In [ ]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [ ]:
chain.invoke("How do I identify a good vintage wine at a restaurant?")

Classification

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

classification_template = PromptTemplate.from_template(
    """You are good at classifying a question.
    Given the user question below, classify it as either being about `Car`, `Restaurant`, or `Technology`.

    <If the question is about car mechanics, models, or automotive technology, classify it as 'Car'>
    <If the question is about cuisines, dining experiences, or restaurant services, classify it as 'Restaurant'>
    <If the question is about gadgets, software, or technological trends, classify it as 'Technology'>

    <question>
    {question}
    </question>

    Classification:"""
)

classification_chain = classification_template | ChatOpenAI() | StrOutputParser()

In [ ]:
def prompt_router(input):
    classification = classification_chain.invoke({"question": input["query"]})

    if classification == "Car":
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif classification == "Restaurant":
        print("Using RESTAURANT")
        return PromptTemplate.from_template(restaurant_template)
    elif classification == "Technology":
        print("Using TECHNOLOGY")
        return PromptTemplate.from_template(technology_template)
    else:
        print("Unexpected classification:", classification)
        return None


input_query = {"query": "What are the latest trends in electric cars?"}
prompt = prompt_router(input_query)

In [ ]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [ ]:
chain.invoke("How do I identify a good vintage wine at a restaurant?")

### Tool Calling

In [ ]:
from langchain_core.tools import tool

@tool
def classify_as_car(question: str) -> str:
    """Classifies the question as 'Car' if it is about car mechanics, models, or automotive technology.

    Args:
        question: The input question to classify.
    """
    print("Function 'classify_as_car' was used.")
    return "Car"


@tool
def classify_as_restaurant(question: str) -> str:
    """Classifies the question as 'Restaurant' if it is about cuisines, dining experiences, or restaurant services.

    Args:
        question: The input question to classify.
    """
    print("Function 'classify_as_restaurant' was used.")
    return "Restaurant"


@tool
def classify_as_technology(question: str) -> str:
    """Classifies the question as 'Technology' if it is about gadgets, software, or technological trends.

    Args:
        question: The input question to classify.
    """
    print("Function 'classify_as_technology' was used.")
    return "Technology"

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, ToolMessage

llm = ChatOpenAI()

query = "How do I identify a good vintage wine at a restaurant?"

tools = [classify_as_car, classify_as_restaurant, classify_as_technology]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {
        "classify_as_car": classify_as_car,
        "classify_as_restaurant": classify_as_restaurant,
        "classify_as_technology": classify_as_technology,
    }[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

In [ ]:
llm_with_tools.invoke(messages)